In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('AAPL', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(-1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 3
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl['3 days future return'].apply(lambda val: to_class(val, 3))
aapl['Return class'].head()

Date
2002-02-11    Neutral
2002-02-12    Neutral
2002-02-13        Neg
2002-02-14        Neg
2002-02-15        Neg
Name: Return class, dtype: object

In [17]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', '3 days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

In [18]:
trafos = [(col, StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
trafos += [('Return class', LabelBinarizer())]

In [19]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 ('Run Up 252', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('Beta 63', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('EMA 100', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('SMA 100', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('SMA MOM 100', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('SP500 SMA 100', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('SP500 Vola 63', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('Sharpe 63', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('Vola 63', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('Return class',
  LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False))]

In [20]:
mapper = DataFrameMapper(trafos, df_out=True)

In [ ]:
mappe

In [21]:
aapl_mapped = mapper.fit_transform(aapl)
aapl_mapped.head()

ValueError: Run Up 252: Expected 2D array, got 1D array instead:
array=[-0.42696629 -0.41477273 -0.41899441 ...,  0.46386886  0.45488658
  0.46522782].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.